# Сложные объединения 
## Юнит 5. РАБОТА С БАЗАМИ ДАННЫХ. SQL
### Skillfactory: DSPR-19

### 4.1. UNION

**UNION**   
Мы умеем присоединять кортежи друг к другу путем добавления столбцов одного к другому с помощью JOIN.

К любому результату запроса можно присоединить другой запрос «снизу», если у него такая же структура (одинаковое количество столбцов, данные того же типа). Для этого существует операция UNION.

Существует две версии этого оператора: UNION выводит уникальные записи, UNION ALL не делает ограничений уникальности и присоединяет все кортежи последующих таблиц к предыдущим. UNION ALL работает быстрее и более предпочтителен, если вы уверены в уникальности объединяемых выборок или если вам нужны неуникальные данные.

Синтаксис выглядит следующим образом.

In [ ]:
query 1
union all
query 2
union all

...

query n

Результатом выполнения такого запроса будут все кортежи результата выполнения query_1 и все кортежи query_2 , выведенные в единой выдаче.

Давайте посмотрим на примере. Решим следующую задачу: соберем все знакомые нам объекты из БД в одном запросе с типом и именем и упорядочим их по алфавиту.

In [ ]:
SELECT 
 t.table_name object_name,
 'таблица' object_type
FROM 
 information_schema."tables" t
union all
SELECT 
 c.column_name object_name,
 'столбец' object_type
FROM 
 information_schema.columns c
union  all
select 
 s.schema_name,
 'схема' 
from 
 information_schema.schemata s
order by 1

Каждый из объединяемых запросов имеет два столбца: имя и тип. Оба поля текстовые, названия для столбцов берутся из самого первого запроса.

### Задание 4.1.1
Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имен водителей и производителей грузовиков. Результатом запроса должны быть два столбца (название объекта и тип объекта — city, state, driver, truck). Отсортируйте список по названию объекта, а затем по типу.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
SELECT 
        c.city_name object_name,
        'city' object_type
from city c 
union
SELECT 
        c2.state object_name,
        'state' object_type
from city AS c2 
union
SELECT 
        d.first_name object_name,
        'driver' object_type
from driver d
union
SELECT 
        t.make object_name,
        'truck' object_type
from truck t
order by 1, 2

### 4.2. UNION и ограничения типов данных
Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Предположим, мы хотим вывести список всех id городов и их названий в одном столбце. Попробуйте выполнить запрос ниже.

In [ ]:
select 
 c.city_id
from 
 shipping.city c
union all
select 
 cc.city_name
from 
 shipping.city cc

Вы получите ошибку "org.postgresql.util.PSQLException: ERROR: UNION types integer and text cannot be matched". Она произошла, потому что мы попытались объединить численный и строковый тип в одной колонке, что невозможно. 

Если мы всё же хотим выполнить поставленную задачу, придется привести оба столбца к одному типу данных. Не каждый текст может быть приведен к числу, но каждое число может быть представлено в текстовом формате.

Забежим немного вперед и поговорим о типизации столбцов. Для типизации в Postgres используется следующий синтаксис: column_name :: column_type. Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text. Любой тип данных может быть приведен к текстовому: пользуйтесь этим для соединения разнородных сущностей, но помните, что сортировка текста отличается от сортировки чисел и дат.

Немного подправив запрос, получим финальный результат.

In [ ]:
select 
 c.city_id::text
from 
 shipping.city c
union all
select 
 cc.city_name
from 
 shipping.city cc

### Задание 4.2.1
Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник. Также добавьте столбец с именем водителя и столбец с типом контакта ('phone' или 'zip' в зависимости от типа). Упорядочите список по столбцу с контактными данными по возрастанию, а затем по имени водителя.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select 
         d.zip_code::TEXT 
        ,d.first_name 
        ,'zip' AS type
from driver d 
union
select 
         d.phone 
        ,d.first_name
        , 'phone' AS type
from driver d 
order by 1, 2

### Задание 4.2.2
Выберите наибольшее из значений:  

Ответ: 1000000 

### Задание 4.2.3
(Не забывайте перед решением задачи проверять код в Metabase!)

А теперь с помощью Metabase сравните эти значения, приведенные к текстовому типу данных, и выберите наибольшее.

In [ ]:
SELECT '541' > '-500' AS result
-- true

### 4.3. UNION ALL и промежуточные итоги
Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения общих агрегатов.

Следующий запрос выводит общее население по всем городам и его детализацию до конкретного города. В запросах, объединяемых через UNION, могут использоваться все агрегатные функции, функции группировки и выборки.  

In [ ]:
select  
 c.city_name,
 c.population
from 
 shipping.city c
union all
select 
 'total' total_name,
 sum(c.population) total_population
from 
 shipping.city c
order by 2 desc

### Задание 4.3.1
Напишите запрос, который выводит общее число доставок ('total_shippings'), а также количество доставок в каждый день. Необходимые столбцы: date_period,cnt_shipping. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
SELECT 
s.ship_date::text AS date_period,
count(s.ship_id) as cnt_shipping
FROM shipment s
GROUP BY s.ship_date
UNION 
SELECT 
'total_shippings',
count(s.ship_id) 
FROM shipment s 
ORDER BY 1 desc

### 4.4. Другие способы применения UNION
**UNION как аналог LEFT JOIN**  
UNION также может быть использован для разделения существующей выборки по выполнению того или иного условия.

Например, с помощью UNION можно разметить заполненность телефонного номера у водителей.

In [ ]:
select 
 d.first_name,
 d.last_name,
 'телефон заполнен' phone_info
from 
 shipping.driver d
where phone is not null
union all 
select 
 d.first_name,
 d.last_name,
 'телефон не заполнен' phone_info
from 
 shipping.driver d
where phone is  null

### Задание 4.4.1
Напишите запрос, который выводит два столбца: city_name и shippings_fake. В первом столбце название города, а второй формируется так:

Если в городе было более 10 доставок, вывести количество доставок в этот город как есть.
Иначе вывести количество доставок, увеличенное на 5.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем по имени в алфавитном порядке.

В выводе не должно быть городов, в которые нет доставок.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select 
c.city_name city_name,
count(s.ship_id) shippings_fake
from city c 
join shipment s on s.city_id = c.city_id 
group by c.city_name 
having count(s.ship_id) > 10
union
select 
c.city_name city_name,
count(s.ship_id) + 5  shippings_fake
from city c 
join shipment s on s.city_id = c.city_id 
group by c.city_name 
having count(s.ship_id) <= 10
order by 2 desc, 1

### Ручная генерация
UNION можно использовать для создания справочников прямо в коде запроса. Например, если мы хотим руками ввести какие-то значения и произвести с ними какие-то манипуляции, или дополнить существующую выдачу своими значениями.

Попробуйте в Metabase! Следующий запрос позволяет нам вывести первые три буквы алфавита и их порядковые номера.

In [ ]:
select 'a'::text letter,'1' ordinal_position
union 
select 'b','2' ordinal_position
union 
select 'c','3' ordinal_position

### 4.5. EXCEPT и INTERSECT

**EXCEPT**  
Синтаксические правила для оператора EXCEPT точно такие же, как и для UNION: количество и тип столбцов должны быть одинаковым.



In [ ]:
query 1 
except 
query 2 

Оператор EXCEPT убирает из результата query_1 все кортежи, совпавшие с результатом выполнения query_2.

Мы уже рассматривали, как решить такую задачу с использованием LEFT JOIN, но EXCEPT будет полезен, если у вас много столбцов и вам не хочется прописывать их равенство в условии JOIN.

Также зачастую EXCEPT используется, когда есть два готовых запроса на выборку: один отражает реальные данные, а другой — мошеннические или ошибочные данные. EXCEPT позволить исключить из выборки ненужные кортежи.

### Задание 4.5.1
Выведите список столбцов, которые есть в таблице shipping.shipment, но нет в других таблицах схемы shipping. Отсортируйте столбцы по возрастанию.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select
    c.column_name 
from
    information_schema.columns c
where 
    c.table_name = 'shipment' and c.table_schema = 'shipping'
except
select
    c.column_name 
from
    information_schema.columns c
where 
    c.table_name = 'customer' and c.table_schema = 'shipping'
except
select
    c.column_name 
from
    information_schema.columns c
where 
    c.table_name = 'driver' and c.table_schema = 'shipping'
except
select
    c.column_name 
from
    information_schema.columns c
where 
    c.table_name = 'city' and c.table_schema = 'shipping'
except
select
    c.column_name 
from
    information_schema.columns c
where 
    c.table_name = 'truck' and c.table_schema = 'shipping'
ORDER BY 1

**INTERSECT**  

Синтаксические правила для оператора INTERSECT точно такие же, как для UNION и для EXCEPT: тип и количество столбцов должны быть одинаковыми.



In [ ]:
query 1 
INTERSECT
query 2 

Оператор INTERSECT оставляет из результатов query_1 все кортежи, совпавшие с результатом выполнения query_2.

Оператор INNER JOIN может быть использован вместо INTERSECT, но INSTERSECT будет полезен, если у вас много столбцов и вам не хочется прописывать их равенство в условии JOIN.

INTERSECT удобен простотой внесения правок в код и возможностью разграничить два запроса переносами строк.

### Задание 4.5.2
Выведите список id городов, в которых есть и клиенты, и доставки, и водители.

(Не забывайте перед отправкой кода проверять его работоспособность и соответствие условиям в Metabase!)

In [ ]:
select 
c.city_id 
from city c 
intersect
select
c2.city_id 
from customer c2 
intersect
select
s.city_id 
from shipment s 
intersect
select 
d.city_id 
from driver d 